In [2]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join


## Preprocess the DUC dataset

In [3]:
data_path = "/Users/deshenghu/Dropbox/Dataset_CS6220/DUC2001"
sums_path = "/Users/deshenghu/Dropbox/Dataset_CS6220/DUC2001/Summaries"

In [6]:
#print(listdir(data_path))
print(type(listdir(data_path)[1]))
# listdir(data_path) is a list of files (format is "str") in that directory, e.g., '.DS_Store', 'annotations.txt', 'AP830325-0143'....
# duc_list is the list of names of the executable files in the DUC2001 directory, excluding the .txt files like 'annotations.txt' 
duc_list = [f for f in listdir(data_path) if isfile(join(data_path, f)) and ".txt" not in f]

# summ_list is the list of the names of the files in the "/DUC2001/Summaries" directory
summ_list = [f for f in listdir(sums_path)]
#print(summ_list)

<class 'str'>


In [7]:
filetuple_list = []
for item in duc_list:
    ff = item.lower()
    if ff + ".txt" in summ_list:
        filetuple_list.append((ff, ff+".txt"))
        summ_list.remove(ff + ".txt")
    else:
        ## the files that exist in the "/DUC2001" while not exist in "/DUC2001/Summaries" will be format like('ap830325-0143', '')
        filetuple_list.append((ff, ""))
print(summ_list)
print(filetuple_list[:3])
print("he")
for item in summ_list:
    ## the files that not exist in the "/DUC2001" while exist in "/DUC2001/Summaries" will be format like('', 'ap900928-0054.txt')
    filetuple_list.append(("", item))
    
## filetuple_list will be this form ('wsj920103-0037', 'wsj920103-0037.txt'), the left one is the name of the 
## executable file while the right one is the corresponding txt file
print(filetuple_list[-6:])

print(filetuple_list[:3])

['ap890325-0143.txt', 'ap900322-0200_system.txt', 'ap900928-0054.txt']
[('.ds_store', ''), ('ap830325-0143', ''), ('ap880217-0175', 'ap880217-0175.txt')]
he
[('wsj920103-0037', 'wsj920103-0037.txt'), ('wsj920114-0145', 'wsj920114-0145.txt'), ('wsj920211-0036', 'wsj920211-0036.txt'), ('', 'ap890325-0143.txt'), ('', 'ap900322-0200_system.txt'), ('', 'ap900928-0054.txt')]
[('.ds_store', ''), ('ap830325-0143', ''), ('ap880217-0175', 'ap880217-0175.txt')]


In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from nltk.corpus import stopwords
from string import punctuation
nltk.download('stopwords')
#stops = list(stopwords.words('english')) + list(punctuation) + ["'s", "''", "``"]

# define two kind of lists of word to be filtered, i.e.,  stopword_punc_others and punc_others
stopword_punc_others = list(stopwords.words('english')) + list(punctuation) + ["'s", "''", "``", "--"]

punc_others = ["'s", "''", "``"]+ list(punctuation)

'''
parse_summ_orig(f) is to parse the summaries files by tokenizing the text file and getting rid of the punctuation while
do not stem or get rid of the stopword
'''
def parse_summ_orig(f):
    text = str(f).lower()
    # get the text of the abstract and introduction part
    text = text[11:-4]
    # split with "\\n"
    text = text.split("\\n")
    
    docu = []
    for item in text:
        if item != "":
            docu.append(item)
    
    for i in range(len(docu)):
        docu[i] = docu[i].replace("\\", "")
        docu[i] = " ".join([w for w in word_tokenize(docu[i]) if not w in punc_others])
    doc = " ".join(docu)
    return docu, doc
         
'''
parse_exe_orig(f) is to parse the executable files by tokenizing the text file and getting rid of the punctuation while
do not stem or get rid of the stopword
'''    
def parse_exe_orig(f):
    
    text = str(f).lower()
    # get the number of the poistion in the string where the text start after the tag<text>
    start = text.find("<text>") + 6
    # get the number of the poistion in the string where the text end before the tag</text>
    end = start + text[start:].find("</text>")
    # this the text in between
    text = text[start:end]
    # substitue all the <> tag ( regardless whether there is any character in the <> or not) with ""
    text = re.sub("<.*?>","",text)
    text = text.split("\\n")
    
    docu = []
    for item in text:
        if item != "":
            docu.append(item)
    
    for i in range(len(docu)):
        docu[i] = docu[i].replace("\\","")
        docu[i] = " ".join([w for w in word_tokenize(docu[i]) if not w in punc_others])
    
    doc = " ".join(docu)
    return docu, doc
'''
parse_summ_stem_nostopword(f) is to parse the summaries files by tokenizing the text file and getting rid of the punctuation while
meanwhile, stem and get rid of the stopword in the summaries files
'''          
    
def parse_summ_stem_nostopword(f):
    text = str(f).lower()
    # get the text of the abstract and introduction part
    text = text[11:-4]
    # split with "\\n"
    text = text.split("\\n")
   # a = a.split("\n")
    
    docu = []
    for item in text:
        if item != "":
            docu.append(item)
    # http://www.nltk.org/_modules/nltk/stem/porter.html
    stemmer = PorterStemmer()
    
    for i in range(len(docu)):
        docu[i] = docu[i].replace("\\", "")
        docu[i] = " ".join([w for w in word_tokenize(docu[i]) if not w in stopword_punc_others])
        docu[i] = stemmer.stem(docu[i])
        
    doc = " ".join(docu)
    return docu, doc

'''
parse_exe_stem_nostopword(f) is to parse the executable files by tokenizing the text file and getting rid of the punctuation while
meanwhile, stem and get rid of the stopword in the executable files
'''   

def parse_exe_stem_nostopword(f):
    text = str(f).lower()
    # get the number of the poistion in the string where the text start after the tag<text>
    start = text.find("<text>") + 6
    # get the number of the poistion in the string where the text end before the tag</text>
    end = start + text[start:].find("</text>")
    # this the text in between
    text = text[start:end]
    # substitue all the <> tag ( regardless whether there is any character in the <> or not) with ""
    text = re.sub("<.*?>","",text)
    
    text = text.split("\\n")
   # a = a.split("\n")
    
    docu = []
    for item in text:
        if item != "":
            docu.append(item)
    stemmer = PorterStemmer()
    
    for i in range(len(docu)):
        docu[i] = docu[i].replace("\\","")
        docu[i] = " ".join([w for w in word_tokenize(docu[i]) if not w in stopword_punc_others])
        docu[i] = stemmer.stem(docu[i])
    
    doc = " ".join(docu)
    return docu, doc

'''
read_file(path) is to read the files (either executable files or the txt files) given the path to the file
'''  
def read_file(path):
    res = ''
    if isfile(path):
        with open(path, 'rb') as f:
            res = f.read()
    return res

# doc_datas_tuple is a list of tuples (f_exe_doc, f_summ_doc) which is a document by " ".join the words after both stemming and stopword-removing
doc_datas_tuple = []

# orig_seg_tuple is a list of tuples (f_exe_orig, f_summ_orig),
# here f_exe_orig is a list of sentences splitted/segmented by "\\n" in the exe files while f_summ_orig is a list of sentences splitted/segmented by "\\n"in the summaries files
orig_seg_tuple = []

# stem_seg_tuple is a list of tuples (f_exe_seg,f_summ_seg),
# here f_exe_seg is a list of sentences splitted/segmented by "\\n" in the exe files and stemmed and stop-word-removed
# while f_summ_seg is a list of sentences splitted/segmented by "\\n" in the summaries files and stemmed and stop-word-removed
stem_seg_tuple = []

for file_exe, file_txt in filetuple_list:
  
    # file_exe is like "'wsj920103-0037'", while file_txt is like "wsj920103-0037.txt'"
    f_exe = read_file(join(data_path, file_exe))
   
    #print(f1)
    f_summ = read_file(join(sums_path, file_txt))
    #print(f_exe)
    
    f_exe_seg, f_exe_doc = parse_exe_stem_nostopword(f_exe)
    f_summ_seg, f_summ_doc = parse_summ_stem_nostopword(f_summ)
    
    f_exe_orig, _ = parse_exe_orig(f_exe)
    f_summ_orig, _ = parse_summ_orig(f_summ)
    
    doc_datas_tuple.append((f_exe_doc, f_summ_doc))
    orig_seg_tuple.append((f_exe_orig, f_summ_orig))
    stem_seg_tuple.append((f_exe_seg,f_summ_seg))
    
 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/deshenghu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
orig_seg_tuple[3]

(['thousands of peole',
  'prayed cheered danced beat drums and observed other traditions',
  'today as a total eclipse of the sun darkened a wide area of',
  'indonesia and the southern philippines',
  'the sun was blacked out by the shadow of the moon for up to four',
  'minutes along a 108-mile swath that moved from the indian ocean',
  'across indonesia and the southern philippine island of mindanao',
  'a partial eclipse was visible across a wide area of asia as far',
  'west as india including southeast asia china japan and new',
  'guinea as well as parts of australia and the pacific islands',
  'scientists said the eclipse would end at sunset in the gulf of',
  'alaska',
  'clouds obscured the solar spectacular in jakarta indonesia',
  'hong kong and in the eastern mindanao city of davao where',
  'thousands of tourists and scientists had gathered to observe the',
  'eclipse',
  'but it was visible for about 3 minutes in this city about 650',
  'miles southeast of manila despit

In [11]:
doc_datas_tuple[2]

('coalition members congress announc wednesday plan sue census bureau effort force agency delete illegal aliens count 1990 40 members house joined federation american immigration reform announcing suit would fil thursday u.s. district court pittsburgh spokesmen said news confer group contends including estimated 2 million illegal aliens national head count us distribute seats house representatives cause unfair shifts seats one state anoth census officials say required count everyon u.s. constitution mention citizenship instructs house apportionment based whol number persons residing various states approach upheld federal court similar suit brought immigration reform group 1980 censu nonetheless dan stein immigration reform feder contended illegal aliens allowed part determining political structure united st rep. tom ridge r-pa. said census bureau actu count everyone develop method determin many people illegally country deduct number figures used reapportioning congress rep. jan meyers 

In [12]:
stem_seg_tuple[2]

(['coalition members congress announc',
  'wednesday plan sue census bureau effort',
  'force agency delete illegal aliens count 1990',
  '40 members house joined federation american',
  'immigration reform announcing suit would fil',
  'thursday u.s. district court pittsburgh spokesmen said',
  'news confer',
  'group contends including estimated 2 million',
  'illegal aliens national head count us',
  'distribute seats house representatives cause unfair',
  'shifts seats one state anoth',
  'census officials say required count everyon',
  'u.s. constitution mention citizenship',
  'instructs house apportionment based whol',
  'number persons residing various states approach',
  'upheld federal court similar suit brought',
  'immigration reform group 1980 censu',
  'nonetheless dan stein immigration reform feder',
  'contended illegal aliens allowed part',
  'determining political structure united st',
  'rep. tom ridge r-pa. said census bureau actu',
  'count everyone develop method 

# DUC 

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
#  DUC_exe_summ_tuple[0] will be the list including all the DUC_exe_doc(each doc is a list) while
#  DUC_exe_summ_tuple[1] will be the list including all the DUC_sum_doc(each doc is a list)
#  (zip(*doc_datas_tuple)) is the "zip " type and thus need to be converted into list
DUC_exe_summ_tuple = list(map(list,zip(*doc_datas_tuple)))
print(DUC_exe_summ_tuple[0][2])
print(DUC_exe_summ_tuple[1][2])

vectorizer_exe = TfidfVectorizer(sublinear_tf = True)
print(type(vectorizer_exe))
vectorizer_exe.fit(DUC_exe_summ_tuple[0])

print(type(vectorizer_exe))
DUC_exe = vectorizer_exe.transform(DUC_exe_summ_tuple[0])
#print(DUC_exe[1])
## DUC_exe.shape is (312, 18772)
print(DUC_exe.shape)
## sparse matrix length is ambiguous; use getnnz()247 " or shape[0]
## thus DUC_exe[3] ==DUC_exe[3][0]
print((DUC_exe[3]))
# dic_items will be something like dict_items([('millions', 10920), ('gallons', 7425), 
# ('crude', 4591), ('oil', 11810), ('spilled', 15860)...])
dic_item_exe = vectorizer_exe.vocabulary_.items()
#print(dic_item_exe)

# word_exe will be something like {10920: 'millions', 7425: 'gallons', 4591: 'crude', 
# 11810: 'oil', 15860: 'spilled', 16731: 'tanker'...}
word_exe = {k:v for v,k in dic_item_exe}
#print(word_exe)

vectorizer_sum = TfidfVectorizer(sublinear_tf = True)
vectorizer_sum.fit(DUC_exe_summ_tuple[1])
DUC_summ = vectorizer_sum.transform(DUC_exe_summ_tuple[1])
word_summ = {k:v for v,k in vectorizer_sum.vocabulary_.items()}

coalition members congress announc wednesday plan sue census bureau effort force agency delete illegal aliens count 1990 40 members house joined federation american immigration reform announcing suit would fil thursday u.s. district court pittsburgh spokesmen said news confer group contends including estimated 2 million illegal aliens national head count us distribute seats house representatives cause unfair shifts seats one state anoth census officials say required count everyon u.s. constitution mention citizenship instructs house apportionment based whol number persons residing various states approach upheld federal court similar suit brought immigration reform group 1980 censu nonetheless dan stein immigration reform feder contended illegal aliens allowed part determining political structure united st rep. tom ridge r-pa. said census bureau actu count everyone develop method determin many people illegally country deduct number figures used reapportioning congress rep. jan meyers r-

In [29]:
# from elasticsearch import Elasticsearch
# es = Elasticsearch()
# for i in range(len(DUC_exe_summ_tuple[0])):
#     doc = {
#         "doc_id": i,
#         "doc_text": DUC_exe_summ_tuple[0][i]
#         #"topic_id":str(con_topic_id[i]),
#         #"top_words": ", ".join(topic_word_con[con_topic_id[i]]),
#         #"doc_topic": str(con_topic_prob[i]),
#         #"KL_summary": kl_sum_con[i],
#         #"LDA_summary": lda_sum_con[i]
#     }
#     res = es.index(index = "duc_exe", doc_type = "doc", id = i, body = doc)
    
    

PUT http://localhost:9200/duc_exe/doc/0 [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py", line 166, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=request_headers, **kw)
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages

PUT http://localhost:9200/duc_exe/doc/0 [status:N/A request:0.051s]
Traceback (most recent call last):
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py", line 166, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=request_headers, **kw)
  File "/Users/deshenghu/anaconda3/lib/python3.6/site-packages

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x1a0aea7b00>: Failed to establish a new connection: [Errno 61] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x1a0aea7b00>: Failed to establish a new connection: [Errno 61] Connection refused)

## LDA for DUC

In [144]:
# link: http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#examples-using-sklearn-decomposition-latentdirichletallocation
from sklearn.decomposition import LatentDirichletAllocation as LDA
# example: http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
n_topic = 20
LDA_DUC_exe = LDA(n_components=n_topic)
LDA_DUC_exe.fit(DUC_exe)
#topic_word_exe = []

##  LDA_DUC_exe.components_ is an array with shape(20, 18772),
## it is Variational parameters for topic word distribution. 
## Since the complete conditional for topic word distribution is a Dirichlet, 
## components_[i, j] can be viewed as pseudocount that represents the number of times word j was assigned to topic i.
print(LDA_DUC_exe.components_.shape)

#print(enumerate(LDA_DUC_exe.components_))

## here prob is 
for i, prob in enumerate(LDA_DUC_exe.components_):
    ## prob is type of np.ndarray with shape (18772,), LDA_DUC_exe.components_.shape is (20, 18772)
    ## here is to get the index of the top 10 word in a certain topic
    top = np.argsort(prob)[::-1][:20]
    print("This is Topic %s in exe files(K=20): %s " % (i,[word_exe[j] for j in top] ))
    
    #print(prob.shape)
    #topic_word_exe.append([word_exe[j] for j in top]) 
##    topic_word_exe is a list with a length of 20 
print(len(topic_word_exe))

LDA_DUC_summ = LDA(n_components=n_topic)
LDA_DUC_summ.fit(DUC_summ)
topic_word_sum = []
for i, prob in enumerate(LDA_DUC_summ.components_):
    top = np.argsort(prob)[::-1][:10]
    print("This is Topic %s in summaries files(K=20): %s " % (i,[word_summ[j] for j in top] ))
    
    topic_word_sum.append([word_summ[j] for j in top])

/Users/deshenghu/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


(20, 18772)
This is Topic 0 in exe files(K=20): ['slovenia', 'communist', 'freedom', 'shining', 'said', 'capital', 'ayacucho', 'serbia', 'true', 'republic', 'yugoslavia', 'year', 'state', 'bock', 'called', 'republics', 'ai', 'eclipse', 'feder', 'raid'] 
This is Topic 1 in exe files(K=20): ['birth', 'voyage', 'zealand', 'puffs', 'jar', 'agony', 'extensive', 'rights', 'coalition', 'mesmerized', 'jaguar', 'politicians', 'census', 'disintegration', 'buena', 'ontario', 'academi', 'fantasy', 'kea', 'reservists'] 
This is Topic 2 in exe files(K=20): ['said', 'police', 'pneumonia', 'taylor', 'miss', 'tuberculosis', 'department', 'members', 'engin', 'one', 'new', 'brut', 'near', 'condition', 'hospital', 'enquirer', 'two', 'service', 'city', 'error'] 
This is Topic 3 in exe files(K=20): ['diamonds', 'diamond', 'beers', 'wherever', 'berman', 'even', 'trade', 'introduced', 'gaviria', 'de', 'stricken', 'agr', 'world', 'bombay', 'cartel', 'solely', 'specials', 'sue', 'dampened', 'moses'] 
This is To

/Users/deshenghu/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


This is Topic 0 in summaries files(K=20): ['diamond', 'revived', 'first', 'beers', 'gartner', 'diamonds', 'good', 'de', 'reasons', 'follow'] 
This is Topic 1 in summaries files(K=20): ['208', 'mixture', 'head', 'commence', 'diamonds', 'hazelwood', 'gaetan', 'criminal', 'shun', 'seemingly'] 
This is Topic 2 in summaries files(K=20): ['battleground', 'ltd', 'spires', 'blacked', 'eclipses', 'chilling', 'city', 'par', 'cargoes', 'manufacturers'] 
This is Topic 3 in summaries files(K=20): ['redirect', 'obviously', 'canada', 'sprinters', 'fr', 'stun', 'four', 'surfaces', 'ben', 'network'] 
This is Topic 4 in summaries files(K=20): ['sheep', 'hurricane', 'cattle', 'end', '10', 'disease', 'expected', 'killed', 'say', 'said'] 
This is Topic 5 in summaries files(K=20): ['said', 'would', 'year', 'one', 'people', 'new', 'mr', 'two', 'may', 'last'] 
This is Topic 6 in summaries files(K=20): ['froggy', 'suburban', '70s', 'witnesses', 'steroid', 'gold', 'johnson', 'addition', 'uneasi', 'indictments']

## K=10

In [78]:
# link: http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#examples-using-sklearn-decomposition-latentdirichletallocation
from sklearn.decomposition import LatentDirichletAllocation as LDA
# example: http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
n_topic = 10
LDA_DUC_exe = LDA(n_components=n_topic)
LDA_DUC_exe.fit(DUC_exe)


##  LDA_DUC_exe.components_ is an array with shape(20, 18772),
## it is Variational parameters for topic word distribution. 
## Since the complete conditional for topic word distribution is a Dirichlet, 
## components_[i, j] can be viewed as pseudocount that represents the number of times word j was assigned to topic i.
print(LDA_DUC_exe.components_.shape)

#print(enumerate(LDA_DUC_exe.components_))

## here prob is 
for i, prob in enumerate(LDA_DUC_exe.components_):
    ## prob is type of np.ndarray with shape (18772,), LDA_DUC_exe.components_.shape is (20, 18772)
    ## here is to get the index of the top 10 word in a certain topic
    top = np.argsort(prob)[::-1][:20]
    print("This is Topic %s in exe files(K=10): %s " % (i,[word_exe[j] for j in top] ))
    #print(prob.shape)
    
    #topic_word_exe.append([word_exe[j] for j in top]) 
##    topic_word_exe is a list with a length of 20 
print(len(topic_word_exe))

LDA_DUC_summ = LDA(n_components=n_topic)
LDA_DUC_summ.fit(DUC_summ)

for i, prob in enumerate(LDA_DUC_summ.components_):
    top = np.argsort(prob)[::-1][:20]
    print("This is Topic %s in summary files(K=10): %s " % (i,[word_summ[j] for j in top] ))
    #topic_word_sum.append([word_summ[j] for j in top])

/Users/deshenghu/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


(10, 18772)
This is Topic 0 in exe files(K=10): ['workmen', 'day', 'climatic', 'veneto', 'malone', 'sporting', 'military', 'police', 'plush', 'tightli', 'grinding', 'renovated', 'elbert', 'relatively', 'field', 'anxious', 'petracca', 'md', 'payments', 'respons'] 
This is Topic 1 in exe files(K=10): ['rather', '08', 'lapierre', 'shoot', 'bemoaned', 'frc', 'walk', 'tol', 'suicid', 'mational', 'stressful', 'frequencies', 'right', 'causing', 'diverts', 'gloves', 'vein', 'modified', 'zigas', 'developing'] 
This is Topic 2 in exe files(K=10): ['could', 'enumerated', 'johnson', 'clara', 'broad', 'backcountri', 'medal', 'histor', 'philosophical', 'operated', 'montgomery', 'then', 'pak', 'retention', 'calgary', 'causing', 'dieppe', 'whalen', 'differing', 'canto'] 
This is Topic 3 in exe files(K=10): ['said', 'thomas', 'miss', 'taylor', 'st', 'california', 'court', 'one', 'race', 'rac', 'people', 'along', 'new', 'thoma', 'two', 'census', 'back', 'black', 'america', 'could'] 
This is Topic 4 in e

/Users/deshenghu/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


This is Topic 0 in summary files(K=10): ['diamond', 'beers', 'de', 'diamonds', 'billion', 'year', 'said', 'sales', 'africa', 'rough', 'world', 'cartel', 'earthquake', 'one', 'eclipse', 'prices', 'south', 'million', 'say', 'new'] 
This is Topic 1 in summary files(K=10): ['said', 'crash', 'tuberculosis', 'health', 'aircraft', 'military', 'crashed', 'flight', 'air', 'two', 'jet', 'people', 'would', 'force', 'cases', 'also', 'increase', 'number', '520', 'pilot'] 
This is Topic 2 in summary files(K=10): ['steroids', 'presenting', 'ababa', 'country', 'said', 'olympic', 'state', 'highly', 'year', 'motifs', 'stockbroker', 'libertarian', 'making', 'montreuil', 'acapulco', 'tighter', 'mexico', 'united', 'burns', 'agrario'] 
This is Topic 3 in summary files(K=10): ['tornado', 'quoted', 'stretch', 'massachusett', 'riveted', 'curry', 'study', 'yerxa', 'imag', 'terribly', 'horror', 'orrin', 'planes', 'londoners', 'vilified', 'tapes', 'securities', 'let', 'better', 'jackson'] 
This is Topic 4 in summ

# NMF

In [53]:
from sklearn.decomposition import NMF
NMF_DUC_exe = NMF(n_components = 20).fit(DUC_exe)
for i, prob in enumerate(NMF_DUC_exe.components_):
    top = np.argsort(prob)[::-1][:20]
    print("This is Topic %s: %s" % (i, [word_exe[j] for j in top]))


Topic 0: ['thomas', 'court', 'right', 'rights', 'amendment', 'gun', 'law', 'supreme', 'clarence', 'civil', 'thoma', 'militia', 'constitutional', 'black', 'arms', 'senate', 'constitution', 'bear', 'nomination', 'guns']
Topic 1: ['hurricane', 'hurricanes', 'storm', 'hurrican', 'storms', 'mph', 'tropical', 'atlantic', 'winds', 'gilbert', 'season', 'sheets', 'forecasters', 'gray', 'center', 'hugo', 'caribbean', 'weather', 'pressure', 'atmospheric']
Topic 2: ['taylor', 'pneumonia', 'hospital', 'miss', 'actress', 'doctors', 'sam', 'bacterial', 'elizabeth', 'john', 'chen', 'hospitalized', 'publicist', 'condition', 'infection', 'marina', 'health', 'st', 'april', '58']
Topic 3: ['exxon', 'valdez', 'oil', 'spill', 'cleanup', 'tanker', 'alaska', 'sound', 'ship', 'reef', 'guard', 'spil', 'vessel', 'prince', 'gallons', 'william', 'coast', 'crude', 'barrels', 'captain']
Topic 4: ['tunnel', 'french', 'british', 'rail', 'eurotunnel', 'channel', 'britain', 'project', 'link', 'billion', 'london', 'chunn

In [54]:
NMF_DUC_summ = NMF(n_components = 20).fit(DUC_summ)
for i, prob in enumerate(NMF_DUC_summ.components_):
    top = np.argsort(prob)[::-1][:20]
    print("This is Topic %s: %s" % (i, [word_summ[j] for j in top]))

Topic 0: ['nafta', 'welfare', 'clinton', 'mr', 'reform', 'president', 'administration', 'support', 'bank', 'poverty', 'house', 'congress', 'benefits', 'trade', 'would', 'mexico', 'us', 'jobs', 'recipients', 'democrats']
Topic 1: ['hurricane', 'hurricanes', 'storm', 'hurrican', 'storms', 'winds', 'atlantic', 'gilbert', 'tropical', 'mph', 'season', 'sheets', 'gray', 'forecasters', 'atmospheric', 'hugo', 'center', 'caribbean', 'pressure', 'rainfall']
Topic 2: ['bse', 'disease', 'cow', 'cattle', 'beef', 'sheep', 'spongiform', 'encephalopathy', 'mad', 'british', 'agriculture', 'human', 'farmers', 'bovine', 'britain', 'animal', 'eating', 'bs', 'humans', 'jakob']
Topic 3: ['crash', 'crashed', 'aircraft', 'air', 'jet', 'pilot', 'flight', 'plane', 'force', 'engine', 'military', 'fighter', 'training', 'aboard', 'said', 'killed', 'accident', 'tactical', 'base', 'crew']
Topic 4: ['exxon', 'oil', 'valdez', 'spill', 'cleanup', 'tanker', 'alaska', 'sound', 'ship', 'reef', 'vessel', 'spil', 'prince', 

In [55]:
from sklearn.decomposition import NMF
NMF_DUC_exe = NMF(n_components = 10).fit(DUC_exe)
for i, prob in enumerate(NMF_DUC_exe.components_):
    top = np.argsort(prob)[::-1][:20]
    print("This is Topic %s: %s" % (i, [word_exe[j] for j in top]))

Topic 0: ['mr', 'police', 'would', 'president', 'congress', 'nafta', 'right', 'rights', 'political', 'term', 'support', 'government', 'state', 'thomas', 'black', 'welfare', 'said', 'law', 'court', 'public']
Topic 1: ['hurricane', 'storm', 'hurricanes', 'hurrican', 'mph', 'winds', 'storms', 'tropical', 'atlantic', 'gilbert', 'florida', 'season', 'sheets', 'hugo', 'forecasters', 'center', 'gray', 'said', 'weather', 'caribbean']
Topic 2: ['taylor', 'pneumonia', 'hospital', 'doctors', 'miss', 'actress', 'sam', 'bacterial', 'elizabeth', 'john', 'chen', 'infection', 'hospitalized', 'health', 'publicist', 'condition', 'marina', 'st', 'april', '58']
Topic 3: ['exxon', 'oil', 'valdez', 'spill', 'cleanup', 'tanker', 'alaska', 'sound', 'ship', 'reef', 'guard', 'prince', 'spil', 'vessel', 'coast', 'gallons', 'william', 'crude', 'captain', 'barrels']
Topic 4: ['diamond', 'beers', 'de', 'diamonds', 'market', 'billion', 'world', 'london', 'tunnel', 'rough', 'african', 'africa', 'mr', 'selling', 'sale

In [56]:
NMF_DUC_summ = NMF(n_components = 10).fit(DUC_summ)
for i, prob in enumerate(NMF_DUC_summ.components_):
    top = np.argsort(prob)[::-1][:20]
    print("This is Topic %s: %s" % (i, [word_summ[j] for j in top]))

Topic 0: ['police', 'mr', 'would', 'president', 'rights', 'congress', 'right', 'thomas', 'political', 'government', 'term', 'black', 'court', 'support', 'state', 'law', 'limits', 'gun', 'nafta', 'public']
Topic 1: ['hurricane', 'storm', 'hurricanes', 'storms', 'winds', 'mph', 'hurrican', 'tropical', 'atlantic', 'gilbert', 'florida', 'season', 'hugo', 'sheets', 'forecasters', 'gray', 'center', 'atmospheric', 'coast', 'caribbean']
Topic 2: ['taylor', 'pneumonia', 'doctors', 'hospital', 'actress', 'miss', 'elizabeth', 'hospitalized', 'sam', 'bacterial', 'infection', 'john', 'viral', 'health', 'marina', '58', 'st', 'condition', 'yeast', 'april']
Topic 3: ['crash', 'crashed', 'aircraft', 'air', 'jet', 'flight', 'plane', 'pilot', 'force', 'military', 'said', 'killed', 'engine', 'aboard', 'training', 'fighter', 'spokesman', 'accident', 'united', 'crew']
Topic 4: ['disease', 'bse', 'cow', 'cattle', 'cases', 'beef', 'health', 'spongiform', 'sheep', 'encephalopathy', 'mad', 'tuberculosis', 'huma

# 20NG news

In [140]:
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
import string

# extracts the archive contents in the ~/scikit_learn_data/20news_home folder 
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')


# from http://scikit-learn.org/stable/datasets/twenty_newsgroups.html
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, stop_words = 'english')

vectors_train = vectorizer.fit(newsgroups_train.data)
# the test data set don't need to fit

## vocabulary_ return a dict, which is a mapping of terms to feature indices.
news_index_word = {k:v for v,k in vectors_train.vocabulary_.items()}
#print(news_index_word)

## news_data is a sparse matrix with a shape(11314, 129791)
news_data = vectors_train.transform(newsgroups_train.data)
print(type(news_data))
## the shape of news_data[0] is (1, 129791), and news_data[0][0]= news_data[0]
print(news_data[0].shape)
print(news_data[0])




## orig_20NG_newsnews_orig is a list of original doc ( each doc is a list of sentences already removing tags(i.e., {},<>,()) and split by ".")
orig_20NG_news = []
for i in range(len(newsgroups_train.data)):
    ## this is to substitue all the <>,{},() tag ( regardless whether there is any character in the tags or not) with ""
    text = re.sub("{.*?}","",newsgroups_train.data[i])
    text = re.sub("<.*?>","",text)
    text = re.sub("(.*?)","",text)
    
    orig_20NG_news.append([item for item in text.replace("\n","").split(".") if item != ""])

## docs_20NG_news is a list of docs ( each doc is ". "join by sentences after removing stopwrods_punctuation  )
docs_20NG_news = []
for i in range(len(newsgroups_train.data)):
    sente = []
    for line in orig_20NG_news[i]:
        sente.append(" ".join([word for word in word_tokenize(line) if word not in stopword_punc_others])) 
    docs_20NG_news.append(". ".join(sente))
    

vec_news = TfidfVectorizer()
news = []
for i in range(len(orig_20NG_news)):
    news.append(". ".join(orig_20NG_news[i]))
# news is a list with len 11314
print(type(news))
print(news[0])

vec_news.fit(news)




<class 'scipy.sparse.csr.csr_matrix'>
(1, 129791)
  (0, 127716)	0.0742991927305
  (0, 124623)	0.108865878685
  (0, 122883)	0.269466576714
  (0, 118710)	0.0454530347448
  (0, 118009)	0.219828187306
  (0, 114435)	0.0761604248544
  (0, 114191)	0.0675447884203
  (0, 113751)	0.216832290906
  (0, 109351)	0.132479489316
  (0, 109041)	0.132914292691
  (0, 108030)	0.0922397821639
  (0, 105904)	0.12096469188
  (0, 104606)	0.103721491694
  (0, 101172)	0.100147479491
  (0, 99616)	0.0694500844506
  (0, 98745)	0.196938829101
  (0, 95941)	0.1326958845
  (0, 94959)	0.0422485559571
  (0, 91882)	0.121498321924
  (0, 87449)	0.0437198779695
  (0, 86414)	0.161255279083
  (0, 83101)	0.108397849009
  (0, 79517)	0.134037890394
  (0, 78807)	0.0734124290528
  (0, 76576)	0.0846748646992
  :	:
  (0, 75214)	0.367128982002
  (0, 74551)	0.112989505049
  (0, 73060)	0.0524663263425
  (0, 67022)	0.0896344759389
  (0, 65967)	0.1199324944
  (0, 63969)	0.0434123241782
  (0, 63237)	0.102249759148
  (0, 57202)	0.19103829020

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [117]:
print(news[1])

From: guykuo@carson. u. washington. edu (Guy Kuo)Subject: SI Clock Poll - Final CallSummary: Final call for SI clock reportsKeywords: SI,acceleration,clock,upgradeArticle-I. D. : shelley. 1qvfo9INNc3sOrganization: University of WashingtonLines: 11NNTP-Posting-Host: carson. u. washington. eduA fair number of brave souls who upgraded their SI clock oscillator haveshared their experiences for this poll.  Please send a brief message detailingyour experiences with the procedure.  Top speed attained, CPU rated speed,add on cards and adapters, heat sinks, hour of usage per day, floppy diskfunctionality with 800 and 1. 4 m floppies are especially requested. I will be summarizing in the next two days, so please add to the networkknowledge base if you have done the clock upgrade and haven't answered thispoll.  Thanks. Guy Kuo 


In [118]:
docs_20NG_news[0]

'From lerxst wam. umd. edu thing Subject WHAT car Nntp-Posting-Host rac3. wam. umd. eduOrganization University Maryland College ParkLines 15 I wondering anyone could enlighten car I sawthe day. It 2-door sports car looked late 60s/early 70s. It called Bricklin. The doors really small. In addition front bumper separate rest body. This I know. If anyone tellme model name engine specs yearsof production car made history whatever info youhave funky looking car please e-mail. Thanks IL brought neighborhood Lerxst'

In [119]:
docs_20NG_news[1]

"From guykuo carson. u. washington. edu Guy Kuo Subject SI Clock Poll Final CallSummary Final call SI clock reportsKeywords SI acceleration clock upgradeArticle-I. D. shelley. 1qvfo9INNc3sOrganization University WashingtonLines 11NNTP-Posting-Host carson. u. washington. eduA fair number brave souls upgraded SI clock oscillator haveshared experiences poll. Please send brief message detailingyour experiences procedure. Top speed attained CPU rated speed add cards adapters heat sinks hour usage per day floppy diskfunctionality 800 1. 4 floppies especially requested. I summarizing next two days please add networkknowledge base done clock upgrade n't answered thispoll. Thanks. Guy Kuo"

## LDA

In [142]:
n_topic = 20
LDA_20NG_news = LDA(n_components=n_topic)
LDA_20NG_news.fit(news_data)
topic_word_news = []
for i, prob in enumerate(LDA_20NG_news.components_):
    top = np.argsort(prob)[::-1][:20]
    topic_word_news.append([news_index_word[j] for j in top])
    # topic_word_sum.append([word_summ[j] for j in top])
    
    print("This is Topic %s: %s " % (i,[news_index_word[j] for j in top] ))
    

/Users/deshenghu/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


This is Topic 0: ['nthu', 'uz', 'ax', 'gtonwu', 'hannover', 'sy', 'tg', 'kh', 'nv', 'xga', 'suresh', 'kk', 'oy', 'svein', 'o2', 'updating', 'ao', 'cx', 'p7', 'regression'] 
This is Topic 1: ['ttu', 'corvette', 'liftoff', 'schnopia', 'jasmine', 'as010b', 'nodine', 'ttacs1', 'blakey', 'ekdfc', 'peon', 'etrat', 'ad354', 'youngblood', 'simchik', 'tcorkum', 'dup', 'c5jlwx', '4h9', 'c5l0v1'] 
This is Topic 2: ['8514', 'spdc', 'auckland', 'uafhp', 'alavi', 'ssa', 'malamas', 'ccastco', '7909', '____________________________________________________________', 'xon', 'vedas', '8063', 'irl', 'constantinos', 'rama', 'kalki', 'argus', 'indo', 'nacjack'] 
This is Topic 3: ['techbook', 'phys', 'stamber', 'zbib', 'actrix', 'amir', 'yassin', 'spk', 'deir', 'k_mullin', 'replica', 'ankara', 'keefe', 'kai', 'goltz', 'mully', 'tartar', 'rosenblatt', 'dortmund', '763'] 
This is Topic 4: ['egan', 'usq', 'phony25', 'lesc', 'carel', 'hellgate', 'tahko', 'genrad', 'goldwing', 'suutari', 'bro', 'blender', 'dfegan'

## NMF

In [72]:
NMF_20NG_news = NMF(n_components = 20).fit(news_data)
for i, prob in enumerate(NMF_20NG_news.components_):
    top = np.argsort(prob)[::-1][:20]
    print("This is Topic %s: %s" % (i, [news_index_word[j] for j in top]))

This is Topic 0: ['god', 'jesus', 'bible', 'christians', 'christian', 'believe', 'people', 'faith', 'christ', 'say', 'church', 'think', 'christianity', 'life', 'does', 'rutgers', 'don', 'truth', 'religion', 'true']
This is Topic 1: ['windows', 'file', 'thanks', 'window', 'files', 'program', 'dos', 'ac', 'uk', 'help', 'use', 'using', 'graphics', 'university', 'version', 'mail', 'ftp', 'problem', 'ms', 'running']
This is Topic 2: ['team', 'game', 'ca', 'games', 'hockey', 'players', 'play', 'year', 'season', 'win', 'nhl', 'league', 'teams', 'toronto', 'baseball', 'player', 'university', 'detroit', 'cup', 'leafs']
This is Topic 3: ['clipper', 'key', 'chip', 'encryption', 'escrow', 'keys', 'crypto', 'algorithm', 'secure', 'government', 'secret', 'security', 'nsa', 'wiretap', 'privacy', 'public', 'des', 'phone', 'encrypted', 'pgp']
This is Topic 4: ['pitt', 'geb', 'gordon', 'banks', 'cs', 'n3jxp', 'chastity', 'dsl', 'cadre', 'shameful', 'skepticism', 'intellect', 'surrender', 'pittsburgh', '

# Extra sum

In [143]:
##DUC_exe = vectorizer_exe.transform(DUC_exe_summ_tuple[0])

## DUC_exe is a sparse matrix  with shape (312, 18772), thus DUC_exe[3] ==DUC_exe[3][0]

from collections import Counter
exe_topic_id = []
exe_topic_prob = []
for i in range(DUC_exe.shape[0]):
    # prob is an array with a shape (20,) like [ 0.00454917  0.00454917  0.00454917  0.00454917  0.00454917  0.00454917 ...]
    prob = LDA_DUC_exe.transform(DUC_exe[i])[0]
    #print(prob)
    #print(np.sum(prob))
    #print(LDA_DUC_exe.components_)
   # print(LDA_DUC_exe.components_.shape)
   # print(type(prob))
    #print(prob)
    # Returns the indices that would sort an array, np.argsort(LDA_DUC_exe.transform(DUC_exe[i])) will return something
    # like [[ 1 19  6  3 15 11 14  7  4  9  0 12 18 16 10 17  2  8  5 13]], [:: -1] will return an inverse list, thus a list of indexes of the largest to the smallest
    arg = np.argsort(LDA_DUC_exe.transform(DUC_exe[i]))[0][::-1]
    
    ## arg.shape is (10,)
  
    exe_topic_id.append(arg[0])
    ## arg[:5][i] is to get the indexes for the top 5 topics and prob[arg[:5][i]] is to get the corresponding prob for that topic
    exe_topic_prob.append({arg[:5][i]:prob[arg[:5][i]] for i in range(5)})
    
    
summs_topic_id = []
summs_topic_prob = []
for i in range(DUC_summ.shape[0]):
    prob = LDA_DUC_summ.transform(DUC_summ[i])[0]
    arg = np.argsort(LDA_DUC_summ.transform(DUC_summ[i]))[0][::-1]
    summs_topic_id.append(arg[0])
    summs_topic_prob.append({arg[:5][i]:prob[arg[:5][i]] for i in range(5)})
print(summs_topic_id)
    
    
    
    
news_topic_id = []
news_topic_prob = []
for i in range(news_data.shape[0]):
    prob = LDA_20NG_news.transform(news_data[i])[0]
    arg = np.argsort(LDA_20NG_news.transform(news_data[i]))[0][::-1]
    news_topic_id.append(arg[0])
    news_topic_prob.append({arg[:5][i]:prob[arg[:5][i]] for i in range(5)})
    
print(news_topic_id)
    
    

'''
def get_KLscore() is to calculate the kl_score for each sentence
paramaters:
   sen_words: a list of words which originate from a sentences after stemming/stop-word-removing
   doc_count: a dict for counting hashable objects, here the key is the word in the document while the value is the count for that word
return:
   kl_score
'''
    
def get_KLscore(sen_words, doc_count):
    length_sen = len(sen_words)
    length_doc = sum(doc_count.values())
    sen_count = Counter(sen_words)
    
    kl_score = 0
    for item in sen_count.keys():
        p = doc_count[item]/length_doc
        q = sen_count[item]/length_sen
        kl_score += p * np.log(p / q)
    return kl_score

'''
def get_KLsum() is to get the summary for a document using KL_sum metric 
paramaters:
   orig: orig_seg_tuple[i][0],  a list of sentences splitted/segmented by "\\n" in the exe files, no stemming or stopword-removing
   doc: stem_seg_tuple[i][0],  list of sentences splitted/segmented by "\\n" in the exe files and stemmed and stop-word-removed
   L: the lenght of the summary
return:
   summary for a document using KL_sum metric
'''

def get_KLsum(orig, doc, L):
    doc_words = [word for line in doc for word in line.split(" ")]
    doc_count = Counter(doc_words)
    #print(doc_count)
    sentences = [line.split(" ") for line in doc]
    sen_score = [get_KLscore(sent,doc_count) for sent in sentences]
    
    pos = []
    l = 0
    #print(sen_score)
    for i in np.argsort(sen_score):
        l += len(orig[i].split(" "))
        
        pos.append(i)
        if l > L:
            break
    ## here is to sort the index for the sentences and thus make the sentences for the summaries are in the orders match the original one        
    pos = sorted(pos)
    #print(pos)
    summ = []
    summ = [orig[i] for i in pos]
    
    return ". ".join(summ)

kl_sum_exe = []


# doc_datas_tuple is a list of tuples (f_exe_doc, f_summ_doc) which is a document by " ".join the words after both stemming and stopword-removing


# orig_seg_tuple is a list of tuples (f_exe_orig, f_summ_orig),
# here f_exe_orig is a list of sentences splitted/segmented by "\\n" in the exe files while f_summ_orig is a list of sentences splitted/segmented by "\\n"in the summaries files


# stem_seg_tuple is a list of tuples (f_exe_seg,f_summ_seg),
# here f_exe_seg is a list of sentences splitted/segmented by "\\n" in the exe files and stemmed and stop-word-removed
# while f_summ_seg is a list of sentences splitted/segmented by "\\n" in the summaries files and stemmed and stop-word-removed

print(len(orig_seg_tuple[0]))
for i in range(len(orig_seg_tuple)):
    if orig_seg_tuple[i][0] != "":
        kl_sum_exe.append(get_KLsum(orig_seg_tuple[i][0], stem_seg_tuple[i][0], 50))
    else:
        kl_sum_exe.append("")  
        
        
kl_sum_summ = []
for i in range(312):
    if orig_seg_tuple[i][0] != "":
        kl_sum_summ.append(get_KLsum(orig_seg_tuple[i][1], stem_seg_tuple[i][1], 100))
    else:
        kl_sum_summ.append("") 
        
## this part is to generate the summaries for the 20 NG news data set based on KL_sum metric  
kl_sum_20NGnews = []

print(type(orig_20NG_news[i]))
print(type(docs_20NG_news[i]))

for i in range(len(orig_20NG_news)):
    if orig_20NG_news[i] != "":
        kl_sum_20NGnews.append(get_KLsum(orig_20NG_news[i], docs_20NG_news[i].split(". "), 50))
    else:
        kl_sum_20NGnews.append("") 
  

[19, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 19, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17

<class 'list'>
<class 'str'>


In [111]:
kl_sum_20NGnews

['eduOrganization: University of Maryland, College ParkLines: 15 I was wondering if anyone out there could enlighten me on this car I sawthe other day.  If anyone can tellme a model name, engine specs, yearsof production, where this car is made, history, or whatever info youhave on this funky looking car, please e-mail',
 "edu (Guy Kuo)Subject: SI Clock Poll - Final CallSummary: Final call for SI clock reportsKeywords: SI,acceleration,clock,upgradeArticle-I.  Top speed attained, CPU rated speed,add on cards and adapters, heat sinks, hour of usage per day, floppy diskfunctionality with 800 and 1. I will be summarizing in the next two days, so please add to the networkknowledge base if you have done the clock upgrade and haven't answered thispoll",
 'i\'m looking into picking up a powerbook 160 or maybe 180 and have a bunchof questions that (hopefully) somebody can answer:* does anybody know any dirt on when the next round of powerbookintroductions are expected?  i\'d heard the 185c was 

In [85]:
kl_sum_exe

['',
 'what could happen with oil traffic in the sound said dennis. wieliczkiewicz said a boom was placed around the exxon valdez. kelso said alaska has a plan for oil spills that calls for. ott said the spill had polluted prince william sound primary. the coast guard said the exxon valdez struck the reef when it',
 'force the agency to delete illegal aliens from its count in 1990. thursday in u.s. district court in pittsburgh spokesmen said at a. more illegal aliens in the national head count which is used to. rep. tom ridge r-pa. said the census bureau should actually. census form asking whether respondents are u.s. citizerns',
 'today as a total eclipse of the sun darkened a wide area of. a partial eclipse was visible across a wide area of asia as far. scientists said the eclipse would end at sunset in the gulf of. aeronautics said a total eclipse was observed in several parts of. partial eclipse that blocked about 75 percent of the sun surface',
 'flip-flop of one seat from califor

In [86]:
kl_sum_summ


['',
 '',
 'some 40 members of congress have joined with the federation for american immigration reform in announcing that a suit will be filed in an effort to stop the census bureau from counting the estimated 2 million illegal aliens living in the united states rep. tom ridge r-pa. said everyone should be counted but the census bureau should develop a method to determine how many people are illegal and deduct that number from the figures used for reapportioning congress 011 rep. jan meyers r-kan. suggested that a question should be put on the census form asking whether respondents are u.s. citizens. rep. tom ridge r-pa. said the census bureau should actually',
 'multitudes of native peoples tourists and scientists today observed a sun that was blackened for up to four minutes the total eclipse along a 108-mile swath moved from the indian ocean across indonesia and the southern philippine island of mindanao and ended in the gulf of alaska a partial eclipse was visible as far west as i

In [131]:
'''
def get_LDAscore() is to calculate the LDA_score for each sentence
paramaters:
   doc_prob: LDA_DUC_exe.transform(DUC_exe)[i], it is probability distribution of a document belonging to the n-topics 
   sent:  a list of words which originate from a sentences after stemming/stop-word-removing (the sentences are from an exe file in each iteration)
   vocabulary: it is a dict, which is a mapping of terms to feature indices, e.g., {'millions': 10920, 'gallons': 7425,...} 
   components:  LDA_DUC_exe.components_, it is an array with shape(n-topic, num-word-in-a-doc),it is Variational parameters for topic word distribution. 
                Since the complete conditional for topic word distribution is a Dirichlet, components_[i, j] 
                can be viewed as pseudocount that represents the number of times word j was assigned to topic i.
return:
   LDAscore
'''
def get_LDAscore(doc_prob, sent, vocabulary, components):
    num_word = [vocabulary[w] for w in sent if w in vocabulary]
    #print(num_word)
    sen_prob = np.sum([components[:,num] for num in num_word], axis = 0)
    
    doc_pd = doc_prob / np.sum(doc_prob)
    sen_pd = sen_prob / np.sum(sen_prob)
    
    LDAscore = 0
    try:
        iter(sen_prob)
    except TypeError:
        res = 0
    else:
        for p, q in zip(doc_pd, sen_pd):
            LDAscore += p*np.log(p/q)
    return LDAscore



'''
def get_LDAsum() is to get the summary for a document using LDAsum metric 
paramaters:
   doc_prob: LDA_DUC_exe.transform(DUC_exe)[i], it is probability distribution of a document belonging to the n-topics 
   orig: orig_seg_tuple[i][0],  a list of sentences splitted/segmented by "\\n" in the exe files, no stemming or stopword-removing
   doc: stem_seg_tuple[i][0],  list of sentences splitted/segmented by "\\n" in the exe files and stemmed and stop-word-removed
   vocabulary: it is a dict, which is a mapping of terms to feature indices, e.g., {'millions': 10920, 'gallons': 7425,...} 
   components:  LDA_DUC_exe.components_, it is an array with shape(n-topic, num-word-in-a-doc),it is Variational parameters for topic word distribution. 
                Since the complete conditional for topic word distribution is a Dirichlet, components_[i, j] 
                can be viewed as pseudocount that represents the number of times word j was assigned to topic i.
   L: the lenght of the summary
return:
   summary for a document using LDAsum metric
'''


def get_LDAsum(doc_prob, orig, doc, vocabulary, components, L):
    scores = [get_LDAscore(doc_prob, sen.split(" "),vocabulary, components) for sen in doc if len(sen)!=1]
    
    pos = []
    l = 0
    for i in np.argsort(scores):
        l += len(orig[i].split(" "))
        pos.append(i)
        if l > L:
            break
            
    ## here is to sort the index for the sentences and thus make the sentences for the summaries are in the orders match the original one                
    pos = sorted(pos)
    
    summ = []
    summ = [orig[i] for i in pos]
    
    return ". ".join(summ)


lda_sum_exe = []
for i in range(312):
    if orig_seg_tuple[i][0] != "":
        lda_sum_exe.append(get_LDAsum(LDA_DUC_exe.transform(DUC_exe)[i], orig_seg_tuple[i][0], stem_seg_tuple[i][0], vectorizer_exe.vocabulary_, LDA_DUC_exe.components_, 50))
    else:
        lda_sum_exe.append("")

        
lda_sum_summ = []
for i in range(312):
    if orig_seg_tuple[i][0] != "":
        lda_sum_summ.append(get_LDAsum(LDA_DUC_summ.transform(DUC_summ)[i], orig_seg_tuple[i][1], stem_seg_tuple[i][1], vectorizer_sum.vocabulary_, LDA_DUC_summ.components_, 50))
    else:
        lda_sum_summ.append("")
        
        
 ## this part is to generate the summaries for the 20 NG news data set based on LDAsum metric       
        
LDA_20NG_news = LDA(n_components=n_topic)

## vec_news = TfidfVectorizer()
tfidf_20NG_news = vec_news.transform(news)
LDA_20NG_news.fit(tfidf_20NG_news)
LDA_20NG_news.transform(tfidf_20NG_news)
lda_sum_20NGnews = []
doc_pro_dis = LDA_20NG_news.transform(tfidf_20NG_news)
for i in range(len(orig_20NG_news)):
    if orig_20NG_news[i][0] != "":
        lda_sum_20NGnews.append(get_LDAsum(doc_pro_dis[i], orig_20NG_news[i], orig_20NG_news[i], vec_news.vocabulary_, LDA_20NG_news.components_, 50))
    else:
        lda_sum_20NGnews.append("")

/Users/deshenghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars
/Users/deshenghu/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [113]:
lda_sum_20NGnews

['From: lerxst@wam. umd. wam. umd. eduOrganization: University of Maryland, College ParkLines: 15 I was wondering if anyone out there could enlighten me on this car I sawthe other day.  It was called a Bricklin.  The doors were really small.  If anyone can tellme a model name, engine specs, yearsof production, where this car is made, history, or whatever info youhave on this funky looking car, please e-mail',
 'From: guykuo@carson. u. edu (Guy Kuo)Subject: SI Clock Poll - Final CallSummary: Final call for SI clock reportsKeywords: SI,acceleration,clock,upgradeArticle-I. : shelley.  Top speed attained, CPU rated speed,add on cards and adapters, heat sinks, hour of usage per day, floppy diskfunctionality with 800 and 1. 4 m floppies are especially requested',
 'From: twillis@ec. ecn. purdue. * how well does hellcats perform?  ;)thanks a bunch in advance for any info - if you could email, i\'ll post asummary (news reading time is at a premium with finals just around thecorner.  :( )--Tom 

In [93]:
lda_sum_exe

['',
 "what we have here is a major environmental catastrophe said. shoreline will be contaminated he said. he also said four members of the coast guard pacific strike. said. their catches she said. he said one or more blood-alcohol tests were administered after. the grounding but he said he did n't know know how many people were. said",
 'thursday in u.s. district court in pittsburgh spokesmen said at a. the group contends that including the estimated 2 million or. more illegal aliens in the national head count which is used to. shifts of seats from one state to another. rep. tom ridge r-pa. said the census bureau should actually. many people are illegally in the country and them deduct that',
 'scientists said the eclipse would end at sunset in the gulf of. aeronautics said a total eclipse was observed in several parts of. officials said about 20,000 tourists and scientists from around. eclipse the last one expected here for at least 50 years. time of year. police said four people we

In [94]:
lda_sum_summ

['',
 '',
 'introduction. thursday in u.s. district court in pittsburgh spokesmen said at a. the group contends that including the estimated 2 million or. more illegal aliens in the national head count which is used to. shifts of seats from one state to another. many people are illegally in the country and them deduct that. ',
 'introduction. scientists said the eclipse would end at sunset in the gulf of. miles north of new delhi. officials said about 20,000 tourists and scientists from around. eclipse the last one expected here for at least 50 years. time of year. police said four people were injured when a passenger bus. ',
 'introduction. balance one another a population expert said wednesday. flip-flop of one seat from california to georgia said william. minorities he said that deleting the one group and adding in the. the only change he said would be that california would gain. than just one. on the other hand new york would lose three seats states losing. ',
 "introduction. torna

## ES

In [ ]:
kl_sum_summ

In [132]:
from elasticsearch import Elasticsearch
es = Elasticsearch()
for i in range(len(DUC_exe_summ_tuple[0])):
    doc = {
        "doc_id": i,
        "doc_text": DUC_exe_summ_tuple[0][i]
        
    }
    res = es.index(index = "exe", doc_type = "doc", id = i, body = doc)

In [145]:
from elasticsearch import Elasticsearch
es = Elasticsearch()
for i in range(len(DUC_exe_summ_tuple[1])):
    doc = {
        "doc_id": i,
        "doc_text": DUC_exe_summ_tuple[1][i],
        "topic_id":str(summs_topic_id[i]), ##summs_topic_id
        "top_words": ", ".join(topic_word_sum[summs_topic_id[i]]),
        "doc_topic": str(summs_topic_prob[i]),
        "KL_summary": kl_sum_summ[i],
        "LDA_summary": lda_sum_summ[i],
        "test": i+1
    }
    res = es.index(index = "sum", doc_type = "doc", id = i, body = doc)

In [146]:
from elasticsearch import Elasticsearch
es = Elasticsearch()
for i in range(len(newsgroups_train.data)):
    doc = {
        "doc_id": i,
        "doc_text": newsgroups_train.data[i],
        "topic_id": str(news_topic_id[i]),
        "top_words": ", ".join(topic_word_news[news_topic_id[i]]),
        "doc_topic": str(news_topic_prob[i]),
        "KL_summary": kl_sum_20NGnews[i],
        "LDA_summary": lda_sum_20NGnews[i]
    }
    res = es.index(index = "news", doc_type = "doc", id = i, body = doc)

In [30]:
t=[('hello','world d a d'),('how. is ', 'are u a g'),('boy, come to', 'the place')]
s=list(map(list,zip(*t)))

In [31]:
s

[['hello', 'how. is ', 'boy, come to'],
 ['world d a d', 'are u a g', 'the place']]

In [27]:
doc_datas_tuple[2]

('thousands peol prayed cheered danced beat drums observed tradit today total eclipse sun darkened wide area indonesia southern philippin sun blacked shadow moon four minutes along 108-mile swath moved indian ocean across indonesia southern philippine island mindanao partial eclipse visible across wide area asia far west india including southeast asia china japan new guinea well parts australia pacific island scientists said eclipse would end sunset gulf alaska clouds obscured solar spectacular jakarta indonesia hong kong eastern mindanao city davao thousands tourists scientists gathered observ eclips visible 3 minutes city 650 miles southeast manila despite early scattered clouds presid corazon aquino flew witness ev richard fisher high altitude observatory bould colo. said despite scattered clouds scientist confident taken useful pictures eclips jakarta wilson sinambela national institut aeronautics said total eclipse observed several part country including 1,000 tourists south sumat

In [28]:
orig_seg_tuple[2]

(['thousands of peole',
  'prayed cheered danced beat drums and observed other traditions',
  'today as a total eclipse of the sun darkened a wide area of',
  'indonesia and the southern philippines',
  'the sun was blacked out by the shadow of the moon for up to four',
  'minutes along a 108-mile swath that moved from the indian ocean',
  'across indonesia and the southern philippine island of mindanao',
  'a partial eclipse was visible across a wide area of asia as far',
  'west as india including southeast asia china japan and new',
  'guinea as well as parts of australia and the pacific islands',
  'scientists said the eclipse would end at sunset in the gulf of',
  'alaska',
  'clouds obscured the solar spectacular in jakarta indonesia',
  'hong kong and in the eastern mindanao city of davao where',
  'thousands of tourists and scientists had gathered to observe the',
  'eclipse',
  'but it was visible for about 3 minutes in this city about 650',
  'miles southeast of manila despit

In [29]:
stem_seg_tuple[2]

(['thousands peol',
  'prayed cheered danced beat drums observed tradit',
  'today total eclipse sun darkened wide area',
  'indonesia southern philippin',
  'sun blacked shadow moon four',
  'minutes along 108-mile swath moved indian ocean',
  'across indonesia southern philippine island mindanao',
  'partial eclipse visible across wide area asia far',
  'west india including southeast asia china japan new',
  'guinea well parts australia pacific island',
  'scientists said eclipse would end sunset gulf',
  'alaska',
  'clouds obscured solar spectacular jakarta indonesia',
  'hong kong eastern mindanao city davao',
  'thousands tourists scientists gathered observ',
  'eclips',
  'visible 3 minutes city 650',
  'miles southeast manila despite early scattered clouds presid',
  'corazon aquino flew witness ev',
  'richard fisher high altitude observatory bould',
  'colo. said despite scattered clouds scientist',
  'confident taken useful pictures eclips',
  'jakarta wilson sinambela nati

In [10]:
print(doc[:1])

[(['millions gallons crude oil', 'spilled tanker ran aground spread across wildlife-rich', 'stretch ocean saturday alaska chief environmental offic', 'criticized cleanup efforts slow', 'biggest oil spill u.s. history created slick', 'seven miles long seven miles wide prince william sound', 'coast guard said reef island western edge bligh', 'island touched slick', 'situation think everyone secret nightmar', 'could happen oil traffic sound said denni', 'kelso commissioner alaska department environment', 'conserv', '240,000 barrels 10,080,000 gallons crude oil', 'alaska north slope spilled early friday 987-foot', 'tanker exxon valdez ran hard aground bligh reef 25 mil', 'outside valdez taken total cargo 1.2 million', 'barrels initial reports indicated 270,000 barrels spil', 'major environmental catastrophe said', 'one oil spill expert richard golob boston publisher golob', 'oil pollution bulletin', 'golob said cleanup equipment site grossli', 'inadequate added even ideal circumstances cle

In [17]:
print(datas_tuple[3])

("two sides trying force chang 1990 census get way results would nearli balance one another population expert said wednesday census bureau pressure exclude illegal alien national head count traditionally counts everyon living countri groups filed suit ignore aliens contend larg concentrations could result states gaining seat house representatives expense st meanwhile groups want final census tot increased account people may overlooked censu often blacks hispanics living urban area stake 435 seats house distribut among states basis popul sides get way change would flip-flop one seat california georgia said william o'hare director policy studies independent popul reference bureau o'hare told breakfast briefing northeast midwest members congress estimates region lose 14 house seats following 1990 census would continue trend evident last several decades not using estimates number illegal aliens undercount minorities said deleting one group ad would make little difference long run change sa

In [9]:
print(orig[:1])

[(['millions of gallons of crude oil that', 'spilled when a tanker ran aground spread across a wildlife-rich', 'stretch of ocean saturday and alaska chief environmental officer', 'criticized cleanup efforts as too slow', 'the biggest oil spill in u.s. history created a slick about', 'seven miles long and seven miles wide in prince william sound the', 'coast guard said only reef island and the western edge of bligh', 'island had been touched by the slick', 'this situation i think was everyone secret nightmare about', 'what could happen with oil traffic in the sound said dennis', 'kelso commissioner of the alaska department of environmental', 'conservation', 'some 240,000 barrels about 10,080,000 gallons of crude oil', 'from alaska north slope spilled early friday when the 987-foot', 'tanker exxon valdez ran hard aground on bligh reef about 25 miles', 'outside valdez where it had taken on a total cargo of 1.2 million', 'barrels initial reports indicated 270,000 barrels had spilled', 'wha

In [35]:
def word_pre(file, summary = False, orig = False):
    print("enter %s" % file)
    if summary:
        print("this is summary")
        if orig:
            print("test")
    elif orig:
        print("this is orig")
    else:
        print("end")
        

In [30]:
word_pre("s")

enter s
end


In [31]:
word_pre("s", True)

enter s
this is summary


In [32]:
word_pre("s", True, False)

enter s
this is summary


In [33]:
word_pre("s", False, True)

enter s
this is orig


In [36]:
word_pre("s", True, True)

enter s
this is summary
test


In [66]:
import re
a= "sss</DATELINE>\n</TEXT>\nsss<>helo\nhellosb   A coali\n"
a = re.sub("<.*?>","",a)
print(type(a))
print(a)
a = a.split("\\n")
print(a)


<class 'str'>
sss

ssshelo
hellosb   A coali

['sss\n\nssshelo\nhellosb   A coali\n']


In [69]:
a = "method to determine how\nmany people are illegally in the country, and them deduct that\nnumber from the figures used for reapportioning Congress.\n   Rep. Jan Meyers, R-Kan., suggested including a question on the\nCensus form asking whether respondents are U.S. citizerns.\n</TEXT>"
a = re.sub("<.*?>","",a)
print(a)
a = a.split("\n")
print(a)

method to determine how
many people are illegally in the country, and them deduct that
number from the figures used for reapportioning Congress.
   Rep. Jan Meyers, R-Kan., suggested including a question on the
Census form asking whether respondents are U.S. citizerns.

['method to determine how', 'many people are illegally in the country, and them deduct that', 'number from the figures used for reapportioning Congress.', '   Rep. Jan Meyers, R-Kan., suggested including a question on the', 'Census form asking whether respondents are U.S. citizerns.', '']


In [71]:
t = "b"Abstract:\nThe unofficial tornado seaso"

<class 'str'>


In [74]:
orig=False
if not orig:
    print("hello")

hello


In [25]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
orig_20NG_news = []
for i in range(len(newsgroups_train.data)):
    a = re.sub("{.*?}","",newsgroups_train.data[i])
    a = re.sub("<.*?>","",a)
    a = re.sub("(.*?)","",a)
    
    orig_20NG_newsnews_orig.append([item for item in a.replace("\n","").split(".") if item != ""])
    
    
docs_20NG_news = []
for i in range(len(newsgroups_train.data)):
    sente = []
    for line in orig_20NG_newsnews_orig[i]:
        sente.append(" ".join([word for word in word_tokenize(line) if word not in stopword_punc_others])) 
    docs_20NG_newsnews_doc.append(". ".join(sente))
    
    
from nltk.stem import PorterStemmer
temmer = PorterStemmer()
for i in range(len(newsgroups_train.data)):
    newsgroups_train.data[i] = " ".join([w for w in word_tokenize(" ".join(orig_20NG_news[i])) if not w in stopword_punc_others])
    newsgroups_train.data[i] = temmer.stem(newsgroups_train.data[i])
vectorizer_news = TfidfVectorizer(sublinear_tf = True, min_df = 5)
vectorizer_news.fit(newsgroups_train.data)
word_news = {k:v for v,k in vectorizer_news.vocabulary_.items()}
data_news = vectorizer_news.transform(newsgroups_train.data) 
print(newsgroups_train.data[0])

from lerxst wam umd edu thing subject what car nntp-posting-host rac3 wam umd eduorganization university maryland college parklines 15 i wondering anyone could enlighten car i sawthe day it 2-door sports car looked late 60s/early 70s it called bricklin the doors really small in addition front bumper separate rest body this i know if anyone tellme model name engine specs yearsof production car made history whatever info youhave funky looking car please e-mail thanks il brought neighborhood lerxst


In [21]:
orig_20NG_news[3]

['From: jgreen@amber (Joe Green)Subject: Re: Weitek P9000 ?Organization: Harris Computer Systems DivisionLines: 14Distribution: worldNNTP-Posting-Host: amber',
 'ssd',
 'csd',
 'harris',
 'comX-Newsreader: TIN [version 1',
 '1 PL9]Robert J',
 'C',
 ' Kyanko (rob@rjck',
 'UUCP) wrote:> abraxis@iastate',
 'edu writes in article :> > Anyone know about the Weitek P9000 graphics chip?> As far as the low-level stuff goes, it looks pretty nice',
 "  It's got this> quadrilateral fill command that requires just the four points",
 "Do you have Weitek's address/phone number?  I'd like to get some informationabout this chip",
 '--Joe Green\t\t\t\tHarris Corporationjgreen@csd',
 'harris',
 'com\t\t\tComputer Systems Division"The only thing that really scares me is a person with no sense of humor',
 '"\t\t\t\t\t\t-- Jonathan Winters']

In [22]:
vec_news = TfidfVectorizer()
news = []
for i in range(len(news_orig)):
    news.append(". ".join(news_orig[i]))
vec_news.fit(news)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [24]:
newsgroups_train.data[0]

'from lerxst wam umd edu thing subject what car nntp-posting-host rac3 wam umd eduorganization university maryland college parklines 15 i wondering anyone could enlighten car i sawthe day it 2-door sports car looked late 60s/early 70s it called bricklin the doors really small in addition front bumper separate rest body this i know if anyone tellme model name engine specs yearsof production car made history whatever info youhave funky looking car please e-mail thanks il brought neighborhood lerxst'

In [91]:
t=['coalition members congress announc','wednesday plan sue census bureau effort', 'force agency delete illegal aliens count 1990','40 members house joined federation american']

In [92]:
doc_words = [word for line in t for word in line.split(" ")]

In [93]:
doc_words

['coalition',
 'members',
 'congress',
 'announc',
 'wednesday',
 'plan',
 'sue',
 'census',
 'bureau',
 'effort',
 'force',
 'agency',
 'delete',
 'illegal',
 'aliens',
 'count',
 '1990',
 '40',
 'members',
 'house',
 'joined',
 'federation',
 'american']

In [94]:
doc_count = Counter(doc_words)

In [95]:
doc_count

Counter({'1990': 1,
         '40': 1,
         'agency': 1,
         'aliens': 1,
         'american': 1,
         'announc': 1,
         'bureau': 1,
         'census': 1,
         'coalition': 1,
         'congress': 1,
         'count': 1,
         'delete': 1,
         'effort': 1,
         'federation': 1,
         'force': 1,
         'house': 1,
         'illegal': 1,
         'joined': 1,
         'members': 2,
         'plan': 1,
         'sue': 1,
         'wednesday': 1})

In [90]:
x = 'blue,red,green'
t=x.split(',')
type(t)

list

In [91]:
t

['blue', 'red', 'green']